In [1]:
from google.colab import files
uploaded = files.upload()

Saving utils.py to utils.py


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy # for tokensizer and preprocessing
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint # bleu is the estimator tels how good the model is
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k # Mukti30k dataset of german to english 
from torchtext.data import Field, BucketIterator # data processing

In [11]:
!python -m spacy download en
!python -m spacy download de
#!python -m spacy download gu
spacy_ger = spacy.load("de")
spacy_eng = spacy.load("en")
#spacy_hin = spacy.load("gu")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [13]:
def tokenizer_ger(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenizer_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

german = Field(tokenize=tokenizer_ger, init_token="<sos>", eos_token="<eos>", lower=True)
english = Field(tokenize=tokenizer_eng, lower=True, init_token="<sos>", eos_token="<eos>")

In [14]:
train_data, valid_data, test_data = Multi30k.splits(exts=(".de",".en"), fields=(german, english))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 852kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 223kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 211kB/s]


In [15]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [27]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out
      

    # def make_src_mask(self, src):
    #   # src shape : (src_len,N)
    #   src_mask = src.transpose(0,1) == self.src_pad_idx
    #   # pytorch transformer shape is (N, src_len)
    #   return src_mask

    # def forward(self, src, trg):
    #   src_seq_length, N = src.shape
    #   trg_seq_length, N = trg.shape

    #   src_position = (torch.arange(0, src_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device)) # unsqueeze(1) add one dimension
    #   trg_position = (torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device))

    #   embed_src = self.dropout((self.src_word_embedding(src)) + self.src_positional_embedding(src_position))
    #   embed_trg = self.dropout((self.trg_word_embedding(trg)) + self.src_positional_embedding(trg_position))

    #   src_padding_mask = self.make_src_mask(src)
    #   trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)

    #   out = self.transformer(
    #       embed_src,
    #       embed_trg,
    #       src_key_padding_mask = src_padding_mask,
    #       tgt_mask = trg_mask
    #   )

    #   return out





In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = False
save_model = True

# training hyperparameter
num_epochs = 5
learning_rate = 3e-4
batch_size = 32

# model hyperparameter
src_vocab_size = len(german.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoding_layers = 3
num_decoding_layers = 3
drop_out = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi["<pad>"]

# tensorboard for nice plots
writer = SummaryWriter("runs/loss_plot")
step = 0

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_sizes=batch_size, sort_within_batch=True, sort_key = lambda x: len(x.src), device=device,)
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoding_layers,
    num_decoding_layers,
    forward_expansion,
    drop_out,
    max_len,
    device,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

if load_model:

  load_checkpoint(torch.load("my_checkpoint.pth.ptar"), model, optimizer)

sentence = "Dieser Junge ist gut im Codieren und er hat die Fledermaus"  # translation :- that boy is good in coding and he has the bat

for epoch in range(num_epochs):
  print(f"[Epoch {epoch} / {num_epochs}]")

  if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

  model.eval()
  translated_sentence = translate_sentence(
  model, sentence, german, english, device, max_length=100
  )

  print(f"Translated example sentence: \n {translated_sentence}")
  model.train()
  losses = []

  for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # Forward prop
        output = model(inp_data, target[:-1, :])

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin.
        # Let's also remove the start token while we're at it
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1
 

score = bleu(test_data,model,german,english,device)
print(f"Bleu score {score*100:.2f}")



[Epoch 0 / 5]
=> Saving checkpoint
Translated example sentence: 
 ['seven', 'seven', 'seven', 'seven', 'seven', 'seven', 'seven', 'hairdo', 'balcony', 'seven', 'seven', 'seven', 'seven', 'convention', 'balcony', 'balcony', 'check', 'harvesting', 'sells', 'sells', 'sells', 'sells', 'sells', 'bill', 'convention', 'convention', 'chewing', 'harvesting', 'balcony', 'balcony', 'bill', 'convention', 'postal', 'convention', 'harvesting', 'balcony', 'balcony', 'balcony', 'harvesting', 'convention', 'seven', 'seven', 'convention', 'convention', 'bill', 'balcony', 'bill', 'balcony', 'bill', 'convention', 'balcony', 'balcony', 'balcony', 'seven', 'harvesting', 'bill', 'balcony', 'balcony', 'squatting', 'bill', 'convention', 'chewing', 'postal', 'convention', 'balcony', 'balcony', 'balcony', 'balcony', 'balcony', 'balcony', 'balcony', 'bill', 'convention', 'balcony', 'balcony', 'balcony', 'balcony', 'balcony', 'bill', 'sells', 'seven', 'convention', 'balcony', 'balcony', 'bill', 'balcony', 'balcony

KeyboardInterrupt: ignored